# Day 7

> **[Peer Session]** [[DAY 07] 경사하강법](https://github.com/boostcamp-ai-tech-4/peer-session/issues/26)

> **[AI Math]** 경사하강법(순한맛)

> **[AI Math]** 경사하강법(매운맛)

### 미분
미분은 변수의 움직임에 따른 함수값의 변화를 측정하기 위한 도구로 최적화에서 제일 많이 사용하는 기법입니다.

$f'(x)=\lim_{h \rightarrow 0}\frac{f(x+h)-f(x)}{h}$

In [1]:
import numpy as np
import sympy as sym
from sympy.abc import x

sym.diff(sym.poly(x**2 + 2*x + 3), x)

Poly(2*x + 2, x, domain='ZZ')

### 경사상승법 / 경사하강법

- 함수 $f$의 주어진 점 $(x,f(x))$에서의 접선의 기울기를 구한다.
- 한 점에서 접선의 기울기를 알면 어느 방향으로 점을 움직여야 함수값이 증가/감소하는지 알 수 있다.

함수값을 증가시키고 싶으면 미분값을 더해주면 되고 이를 경사상승법(gradient ascent)라고 한다.

이는 함수의 극대값의 위치를 구할 때 사용한다.

함수값을 감소시키고 싶으면 미분값을 빼주면 되고 이를 경사하강법(gradient descent)라고 한다.

이는 함수의 극소값의 위치를 구할 때 사용한다.

In [2]:
def func(val):
    fun = sym.poly(x**2 + 2*x + 3)
    return fun.subs(x, val), fun

def func_gradient(fun, val):
    _, function = fun(val)
    diff = sym.diff(function, x)
    return diff.subs(x, val), diff

def gradient_descent(fun, init_point, lr_rate=1e-2, epsilon=1e-5):
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val = val - lr_rate*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    
    print("함수: {}, 연산횟수: {}, 최소점: ({}, {})".format(fun(val)[1], cnt, val, fun(val)[0]))

gradient_descent(fun=func, init_point=np.random.uniform(-2, 2))

함수: Poly(x**2 + 2*x + 3, x, domain='ZZ'), 연산횟수: 654, 최소점: (-0.999995063905607, 2.00000000002436)


벡터가 입력인 다변수 함수의 경우 편미분(partial differentiation)을 사용한다.

각 변수 별로 편미분을 계산한 그레디언트(gradient) 벡터를 이용하여 경사하강/경사상승법에 사용할 수 있다.

$\triangledown f=(\partial_{x_1}f, \partial_{x_2}f, ..., \partial_{x_d}f)$

여기서 $\partial_{x_i}f(\mathrm x)=\lim_{h \rightarrow 0}\frac{f(\mathrm x+he_i)-f(\mathrm x)}{h}$

In [3]:
import sympy as sym
from sympy.abc import x, y

sym.diff(sym.poly(x**2 + 2*x*y + 3) + sym.cos(x + 2*y), x)

C:\Users\kamme\anaconda3\lib\site-packages\sympy\polys\polytools.py:74: SymPyDeprecationWarning: 

Mixing Poly with non-polynomial expressions in binary operations has
been deprecated since SymPy 1.6. Use the as_expr or as_poly method to
convert types instead. See https://github.com/sympy/sympy/issues/18613
for more info.

  SymPyDeprecationWarning(


2*x + 2*y - sin(x + 2*y)

예를 들어 함수 $f(x,y)=x^2+2y^2$이 있을 때

그레디언트 벡터 $\triangledown f(x,y)$는 각 점 $(x,y)$에서 가장 빨리 증가하는 방향으로 흐르게 된다.

$\Rightarrow -\triangledown f$는 $\triangledown (-f)$랑 같고 이는 각 점에서 가장 빨리 감소하게 되는 방향과 같다.

In [4]:
def eval_(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval

def func_multi(val):
    x_, y_ = val
    func = sym.poly(x**2 + 2*y**2)
    return eval_(func, [x_, y_]), func

def func_gradient(fun, val):
    x_, y_ = val
    _, function = fun(val)
    diff_x = sym.diff(function, x)
    diff_y = sym.diff(function, y)
    grad_vec = np.array([eval_(diff_x, [x_, y_]), eval_(diff_y, [x_, y_])], dtype=float)
    return grad_vec, [diff_x, diff_y]

def gradient_descent(fun, init_point, lr_rate=1e-2, epsilon=1e-5):
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, val)
    while np.linalg.norm(diff) > epsilon:
        val = val - lr_rate*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    
    print("함수: {}, 연산횟수: {}, 최소점: ({}, {})".format(fun(val)[1], cnt, val, fun(val)[0]))

pt = [np.random.uniform(-2, 2), np.random.uniform(-2, 2)]
gradient_descent(fun=func_multi, init_point=pt)

함수: Poly(x**2 + 2*y**2, x, y, domain='ZZ'), 연산횟수: 517, 최소점: ([-4.92851299e-06 -7.53247600e-10], 2.42902413865449E-11)


선형회귀의 목적식은 $\parallel \mathrm y-\mathrm X\beta\parallel_2$이고 이를 최소화하는 $\beta$를 찾아야 하므로 다음과 같은 그레디언트 벡터를 구해야 한다.

$\triangledown_\beta \parallel \mathrm y-\mathrm X\beta\parallel_2=(\partial_{\beta_1}\parallel \mathrm y-\mathrm X\beta\parallel_2, ..., \partial_{\beta_d}\parallel \mathrm y-\mathrm X\beta\parallel_2)$

여기서 $\partial_{\beta_d}\parallel \mathrm y-\mathrm X\beta\parallel_2=\partial_{\beta_k}\left\{\frac{1}{n}\sum_{i=1}^n\left(\mathrm y_i-\sum_{j=1}^d\mathrm X_{ij}\beta_{j}\right)^2\right\}^\frac{1}{2}=-\frac{\mathrm X^T_{.k}(\mathrm y-\mathrm X\beta)}{n\parallel \mathrm y-\mathrm X\beta\parallel}_2$ 이므로

$\triangledown_\beta \parallel \mathrm y-\mathrm X\beta\parallel_2=(\partial_{\beta_1}\parallel \mathrm y-\mathrm X\beta\parallel_2, ..., \partial_{\beta_d}\parallel \mathrm y-\mathrm X\beta\parallel_2)=\left(-\frac{\mathrm X^T_{.1}(\mathrm y-\mathrm X\beta)}{n\parallel \mathrm y-\mathrm X\beta\parallel}_2, ..., -\frac{\mathrm X^T_{.d}(\mathrm y-\mathrm X\beta)}{n\parallel \mathrm y-\mathrm X\beta\parallel}_2\right)=-\frac{\mathrm X^T(\mathrm y-\mathrm X\beta)}{n\parallel \mathrm y-\mathrm X\beta\parallel}_2$

이제 목적식을 최소화하는 $\beta$를 구하는 경사하강법 알고리즘은 다음과 같다.

$\beta^{(t+1)}\leftarrow\beta^{(t)}-\lambda\triangledown_\beta\parallel \mathrm y-\mathrm X\beta^{(t)}\parallel$

$\beta^{(t+1)}\leftarrow\beta^{(t)}+\frac{\lambda}{n}\frac{\mathrm X^T(\mathrm y-\mathrm X\beta^{(t)})}{\parallel \mathrm y-\mathrm X\beta^{(t)}\parallel}$

In [5]:
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

beta_gd = [10.1, 15.1, -6.5] # [1, 2, 3] 이 정답
X_ = np.array([np.append(x, [1]) for x in X]) # intercept 항 추가

for t in range(5000):
    error = y - X_ @ beta_gd
    # error = error / np.linalg.norm(error)
    grad = -np.transpose(X_) @ error
    beta_gd = beta_gd - 0.01*grad

print(beta_gd)

[1.00000367 1.99999949 2.99999516]


이론적으로 경사하강법은 미분가능하고 볼록(convex)한 함수에 대해선 적절한 학습률과 학습횟수를 선택했을 때 수렴이 보장되어 있다.

특히 선형회귀의 경우 목적식 $\parallel \mathrm y-\mathrm X\beta\parallel_2$은 회귀계수 $\beta$에 대해 볼록함수이기 때문에 알고리즘을 충분히 돌리면 수렴이 보장된다.

하지만 비선형회귀 문제의 경우 목적식이 볼록하지 않을 수 있으므로 수렴이 항상 보장되지는 않는다.

### 확률적 경사하강법 (SGD)
확률적 경사하강법(stochastic gradient descent)은 모든 데이터를 사용해서 업데이트하는 대신 데이터 한 개 또는 일부(미니배치)를 활용하여 업데이트 한다.

볼록이 아닌(non-convex) 목적식은 SGD를 통해 최적화 할 수 있다.

$\theta^{(t+1)}\leftarrow\theta^{(t)}-\widehat{\triangledown_\theta \mathcal L(\theta^{(t)})}$

$\mathbb{E}\left[ \widehat{\triangledown_\theta \mathcal L}\right]\approx\triangledown_\theta\mathcal L$

SGD는 데이터의 일부를 가지고 파라미터를 업데이트하기 때문에 연산자원을 좀 더 효율적으로 활용하는데 도움이 된다.

- 경사하강법 : $\beta^{(t+1)}\leftarrow\beta^{(t)}+\frac{2\lambda}{n}\mathrm X^T(\mathrm y-\mathrm X\beta^{(t)})$
    - $O(d^2n)$
- 확률적 경사하강법 : $\beta^{(t+1)}\leftarrow\beta^{(t)}+\frac{2\lambda}{b}\mathrm X^T_{(b)}(\mathrm y_{(b)}-\mathrm X_{(b)}\beta^{(t)})$
    - $O(d^2b)$

전체 데이터 $(\mathrm X,\mathrm y)$를 쓰지 않고 미니배치 $(\mathrm X_{(b)},\mathrm y_{(b)})$를 써서 업데이트 하므로 연산량이 $\frac{b}{n}$로 감소한다.

- 경사하강법은 전체데이터 $\mathcal D=(\mathrm X, \mathrm y)$를 가지고 목적식의 그레디언트 벡터인 $\triangledown_\theta\mathcal L(\mathcal D,\theta)$를 계산한다.
- SGD는 미니배치 $\mathcal D_{(b)}=(\mathrm X_{(b)}, \mathrm y_{(b})\subset\mathcal D$를 가지고 그레디언트 벡터를 계산한다.
- 미니배치는 확률적으로 선택하므로 목적식 모양이 바뀌게 된다.
- SGD는 볼록이 아닌 목적식에도 사용 가능하므로 경사하강법보다 머신러닝 학습에 더 효율적이다.